Install/Import/Versions

In [ ]:
! python -m pip install lightkurve --upgrade

In [ ]:
pip install astropy[recommended] --upgrade

In [ ]:
import sys; print("Python", sys.version)
! python -c "import lightkurve; print(lightkurve.__version__)"
import astropy
astropy.__version__

In [ ]:
#Necessary to run
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


Lightcurve search/download

In [ ]:
#Decide target
##########################################################
Star_name = ' '
##########################################################

search_lcf_refined = lk.search_lightcurve(Star_name, author="SPOC", exptime=120)
search_lcf_refined 

In [ ]:
#Show collections
lcf = search_lcf_refined.download_all()
lcf

In [ ]:
#Specific array
lcf[5]

No. of arrays to merge

In [ ]:
##only one array
flat_lca = lcf[ ].flatten(window_length=1001)
Merged_lc = flat_lca.remove_outliers(sigma=6)
ab = Merged_lc.errorbar()
ab.set_title("Merged Clipped & Flattened light curves of " + str(Star_name));

In [ ]:
##only two arrays
flat_lca = lcf[ ].flatten(window_length=1001)
clipped_lca = flat_lca.remove_outliers(sigma=6)
flat_lcb = lcf[ ].flatten(window_length=1001)
clipped_lcb = flat_lcb.remove_outliers(sigma=6)

Merged_lc = clipped_lca.normalize().append(clipped_lcb.normalize())
ab = Merged_lc.errorbar()
ab.set_title("Merged Clipped & Flattened light curves of " + str(Star_name));

In [ ]:
#####auto-merge for all lightcurves in collection#####################################################################
for x in range(1,len(lcf)):
    #lc of x-1
    flat_lca = lcf[x-1].flatten(window_length=1001)
    clipped_lca = flat_lca.remove_outliers(sigma=3)
    #lc of x
    flat_lcb = lcf[x].flatten(window_length=1001)
    clipped_lcb = flat_lcb.remove_outliers(sigma=3)
    if x==1:
        Merged_lc = clipped_lca.normalize().append(clipped_lcb.normalize())
    else:
        Merged_lc = Merged_lc.normalize().append(clipped_lcb.normalize())
        
ab = Merged_lc.errorbar()
ab.set_title("Merged Clipped & Flattened light curves of " + str(Star_name));

Finding Periods

In [ ]:
#BLS periodogram
# Create array of periods to search ###############
period = np.linspace(1,  , 2000 )
# Create a BLSPeriodogram
bls = Merged_lc.to_periodogram(method='bls', period=period, frequency_factor=2000);
bls.plot();

In [ ]:
planet_x_period = bls.period_at_max_power
planet_x_t0 = bls.transit_time_at_max_power
planet_x_dur = bls.duration_at_max_power

# print period
planet_x_period

In [ ]:
#model of a transit from parameters
planet_x_model = bls.get_transit_model(period=planet_x_period,transit_time=planet_x_t0,duration=planet_x_dur)

#plot model and the phasefolded lightcurve
ax = Merged_lc.fold(period=planet_x_period, epoch_time=planet_x_t0).scatter()
planet_x_model.fold(planet_x_period, planet_x_t0).plot(ax=ax, c='r', lw=2)
ax.set_xlim(-0.5,0.5);
#ax.set_ylim(0.99,1.01);

In [ ]:
#get transit statistics
stats = bls.compute_stats(planet_x_period,planet_x_dur, planet_x_t0)
stats

In [ ]:
#Non scatter fold
folded_lc = Merged_lc.fold(period=planet_x_period, epoch_time=planet_x_t0)
folded_lc.plot()
plt.xlim(-2,2);
#Binning to clean image
import astropy.units as u
#######################################################
binned_lc = folded_lc.bin(time_bin_size=5*u.minute)
#######################################################
aax=binned_lc.errorbar()
planet_x_model.fold(planet_x_period, planet_x_t0).plot(ax=aax, c='r', lw=2)
plt.xlim(-0.5,0.5);

Ratio of radii

In [ ]:
##########################################################
#depth of transit from stats
δ = 
δ_err = 
##########################################################
#Radius in units of Rsuns
Rstar_wrtS = 
Rstar_wrtS_err = 
##########################################################
#Constants
Rsun = 6.963e8
Rearth = 6371e3

Rstar = Rstar_wrtS*(Rsun)
frac_Rstar_err = Rstar_wrtS_err/Rstar_wrtS
Rstar_err = (frac_Rstar_err)*(Rstar)

sqrtδ = δ**0.5
frac_sqrtδ_err = (0.5*(δ_err/δ))
frac_Rplanet_err = (((frac_sqrtδ_err)**2)+((frac_Rstar_err)**2))**0.5
Rplanet = (sqrtδ)*(Rstar)
Rplanet_err = frac_Rplanet_err*Rplanet
Rplanet_wrtE = Rplanet/(Rearth)
Rplanet_wrtE_err = (Rplanet_err/Rplanet)*Rplanet_wrtE

Prob ROck

In [ ]:
def probRock (Rp):
    if Rp<= 1.5:
        return 1
    elif Rp > 1.5 and Rp < 2.5:
        return (2.5-Rp)
    else:
        return 0

Output

In [ ]:
#Period
print("depth=",δ*1000000,"ppm")
print("Duration=",bls.duration_at_max_power)
print("Period=",'{:.3}'.format(planet_x_period))
#Radius
#round error to one sig fig and round radius accordingly
import uncertainties
Rp = uncertainties.ufloat(Rplanet, Rplanet_err)
print("Rp=",'{:.1u}'.format(Rp), "m")
#compare to Earth
Rp_wrtE = uncertainties.ufloat(Rplanet_wrtE, Rplanet_wrtE_err)
print("Rp=",'{:.1u}'.format(Rp_wrtE), "Earth radii")
#Get probRock
print("Pr(Rp)=",probRock(Rplanet_wrtE))